In [37]:
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import warnings
warnings.filterwarnings('ignore')

from revolve.grids import MLPParameterGrid
from revolve.architectures import MLPStrategy
from revolve.algorithms import EvolutionaryAlgorithmElitism
from revolve.operators import (
    Operations,
    random_mutation,
    uniform_crossover,
    roulette_wheel_selection,
)


In [38]:
train, test = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)

x_train_full, y_train_full = train
x_test, y_test = test

x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, test_size=0.1, random_state=42)

In [39]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)

In [40]:
operations = Operations()
operations.register(roulette_wheel_selection)
operations.register(uniform_crossover, probability=0.5)
operations.register(random_mutation, probability=0.3)

In [49]:
mlp_params = MLPParameterGrid(
    input_shape=(x_train.shape[1],),
    regression_target=1,
    regression_activation= "relu",
    hidden_neurons = [0, 64, 128, 258, 512, 1024, 2048],
    activation=['relu', 'swish'],
    dropout=[0.1, 0.2, 0.3, 0.4, 0.5],
    batch_size=[16, 32, 64],
    optimizer=['adam', 'sgd', 'rmsprop'],
    learning_rate=[1.0e-4, 7.5e-4, 5e-4, 2.5e-4],
    l1=1e-6,
    l2=1e-6,
)



In [50]:
mlp_params.static_parameters

{'input_shape': (13,),
 'regression_target': 1,
 'regression_activation': 'relu',
 'l1': 1e-06,
 'l2': 1e-06}

In [51]:
mlp_params.learnable_parameters

{'hidden_neurons': [0, 64, 128, 258, 512, 1024, 2048],
 'activation': ['relu', 'swish'],
 'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
 'batch_size': [16, 32, 64],
 'optimizer': ['adam', 'sgd', 'rmsprop'],
 'learning_rate': [0.0001, 0.00075, 0.0005, 0.00025]}

In [52]:
callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=2)

mlp = MLPStrategy(
    max_fc = 3,
    squeeze_fc=False,
    epochs=50,
    callback=callback,
    loss='mean_squared_error',
    metric = tfa.metrics.RSquare(),
    parameters = mlp_params
)

In [53]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
valid_data = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))

data = (train_data, valid_data, test_data)

In [62]:
ea = EvolutionaryAlgorithmElitism(
    strategy=mlp,
    pop_size=10,
    elitism_size=1,
    operations=operations,
)

In [63]:
best_chromosome = ea.fit(
    data,
    generations=10,
)

Generations:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Generation 0,                 Best error: 20.3051700592041,                 Best R2 0.7560950517654419


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Generation 1,                 Best error: 19.31211280822754,                 Best R2 0.7681102752685547


...

Generation 7,                 Best error: 13.591290473937988,                 Best R2 0.8368948698043823


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Generation 8,                 Best error: 13.591290473937988,                 Best R2 0.8368948698043823


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

Generation 9,                 Best error: 13.591290473937988,                 Best R2 0.8368949294090271


In [64]:
df = ea.results_df()
df.head()

,learnt_parameters,static_parameters,loss,metric,generation
0,"{'fc_0': {'hidden_neurons': 1024, 'activation'...","{'input_shape': (13,), 'regression_target': 1,...",26.428553,0.682887,0
1,"{'fc_0': {'hidden_neurons': 1024, 'activation'...","{'input_shape': (13,), 'regression_target': 1,...",25.976439,0.689116,0
2,"{'fc_0': {'hidden_neurons': 2048, 'activation'...","{'input_shape': (13,), 'regression_target': 1,...",103.804733,-0.246576,0
3,"{'fc_0': {'hidden_neurons': 128, 'activation':...","{'input_shape': (13,), 'regression_target': 1,...",38.710682,0.535009,0
4,"{'fc_0': {'hidden_neurons': 64, 'activation': ...","{'input_shape': (13,), 'regression_target': 1,...",35.146130,0.577817,0


In [65]:
model = best_chromosome.decode(mlp_params)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense (Dense)               (None, 258)               3612      
                                                                 
 dropout (Dropout)           (None, 258)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                16576     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 512)               33280     
                                                                 
 dropout_2 (Dropout)         (None, 512)               0     

In [66]:
best_chromosome_row = df[(df.loss == df.loss.min()) & (df.generation == df.generation.max())]

In [67]:
list(best_chromosome_row.learnt_parameters)

[{'fc_0': {'hidden_neurons': 2048,
   'activation': 'swish',
   'dropout': 0.4,
   'l1': 1e-06,
   'l2': 1e-06},
  'fc_1': {'hidden_neurons': 64,
   'activation': 'relu',
   'dropout': 0.4,
   'l1': 1e-06,
   'l2': 1e-06},
  'fc_2': {'hidden_neurons': 1024,
   'activation': 'relu',
   'dropout': 0.5,
   'l1': 1e-06,
   'l2': 1e-06},
  'batch_size_3': {'batch_size': 16},
  'optimizer_4': {'optimizer': 'adam'},
  'learning_rate_5': {'learning_rate': 0.00075}}]

In [68]:
list(best_chromosome_row.static_parameters)

[{'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu',
  'l1': 1e-06,
  'l2': 1e-06}]

In [69]:
elite_models = ea.elite_models
elite_models
